In [ ]:
%load_ext autoreload
%autoreload 2

# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [3]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")


2023/03/03 12:17:20 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'huggingface' with profile: 'local'


In [4]:
r.clean()

In [5]:
r.inspect()

In [6]:
r.run("ingest")

2023/03/03 12:17:27 INFO mlflow.recipes.step: Running step ingest...
Using custom data configuration default-85cf05310b0c27bb
Found cached dataset json (/Users/jin.zhang/.cache/huggingface/datasets/json/default-85cf05310b0c27bb/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)

100%|██████████| 1/1 [00:00<00:00, 873.45it/s]

Creating parquet from Arrow format: 100%|██████████| 8/8 [00:00<00:00, 1228.92ba/s]


name,type
character,string
speech,string
character,speech
First Citizen,"Before we proceed any further, hear me speak."
All,"Speak, speak."
First Citizen,You are all resolved rather to die than to famish?
All,Resolved. resolved.
First Citizen,"First, you know Caius Marcius is chief enemy to the people."


In [7]:
r.run("split")

2023/03/03 12:17:28 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/03/03 12:17:30 INFO mlflow.recipes.step: Running step split...


In [39]:
r.run("transform")

2023/03/03 14:31:16 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


In [23]:
r.run("train")

2023/03/03 13:35:57 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/03/03 13:35:59 INFO mlflow.recipes.step: Running step transform...
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/mlflow-2.1.2.dev0-py3.8.egg/mlflow/recipes/step.py", line 139, in run
    self.step_card = self._run(output_directory=output_directory)
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/mlflow-2.1.2.dev0-py3.8.egg/mlflow/recipes/steps/transform.py", line 163, in _run
    train_transformed.to_parquet(
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/pandas/core/frame.py", line 2976, in to_parquet
    return to_parquet(
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages

MlflowException: Failed to run step 'train' of recipe 'huggingface'. An error was encountered while running step 'transform': Traceback (most recent call last):
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/mlflow-2.1.2.dev0-py3.8.egg/mlflow/recipes/step.py", line 139, in run
    self.step_card = self._run(output_directory=output_directory)
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/mlflow-2.1.2.dev0-py3.8.egg/mlflow/recipes/steps/transform.py", line 163, in _run
    train_transformed.to_parquet(
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/pandas/core/frame.py", line 2976, in to_parquet
    return to_parquet(
  File "/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/pandas/io/parquet.

In [ ]:
r.run("evaluate")

In [ ]:
r.run("register")

In [ ]:
r.inspect("train")

In [ ]:
training_data = r.get_artifact("training_data")
training_data.describe()

In [ ]:
from typing import Dict, Any, List, Tuple
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    DistilBertForMaskedLM,
    Seq2SeqTrainer,
    TrainingArguments,
)


def trainer_fn(estimator_params: Dict[str, Any]):
    """
    Returns an *untrained* HF trainer here.

    Input estimator_params is a dictionary of parameters passed to the estimator.
    It contains the following keys:
      'train_dataset': A ``datasets.Dataset`` object for training.
      'cache_dir': A string containing the path to the cache directory.
    """
    training_args = TrainingArguments(
        output_dir=estimator_params["cache_dir"],
        max_iters=10)
    
    # Model name
    model_name = "distilbert-base-uncased"
    config = AutoConfig.from_pretrained(
        model_name,
        cache_dir=training_args.output_dir,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=training_args.output_dir,
        use_fast=True,
    )
    model = DistilBertForMaskedLM.from_pretrained(
        model_name,
        config=config,
        cache_dir=training_args.output_dir,
    )
    # Global parameters for tokenizer and model.
    max_seq_length = min(128, tokenizer.model_max_length)
    padding="max_length"
    question_column = "character"
    answer_column = "speech"
    
    # We resize the embeddings only when necessary to avoid index errors.
    embedding_size = model.get_input_embeddings().weight.shape[0]
    if len(tokenizer) > embedding_size:
        model.resize_token_embeddings(len(tokenizer))

    def preprocess_squad_batch(
        examples,
        question_column: str,
        answer_column: str,
    ) -> Tuple[List[str], List[str]]:
        questions = examples[question_column]
        answers = examples[answer_column]
        pairs = [[q, a] for q, a in zip(questions, answers)]
        return questions, answers, pairs
    
    def preprocess_examples(examples):
        inputs, targets, pairs = preprocess_squad_batch(
            examples, question_column, answer_column
        )
        model_inputs = tokenizer(
            inputs,
            max_length=max_seq_length,
            padding=padding,
            truncation=True,
            return_tensors="pt",
        )
        labels = tokenizer(
            targets,
            max_length=max_seq_length,
            padding=padding,
            truncation=True,
            return_tensors="pt",
        )
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]
        model_inputs["labels"] = labels["input_ids"]
        model_inputs["decoder_input_ids"] = labels["input_ids"]
        return model_inputs

    train_dataset = estimator_params["train_dataset"]
    # Create train feature from dataset
    train_dataset = train_dataset.map(
        preprocess_examples,
        batched=True,
        num_proc=1,
        load_from_cache_file=True,
        remove_columns=[question_column, answer_column],
        desc="Running tokenizer on train dataset",
    )

    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=tokenizer.pad_token_id,
        pad_to_multiple_of=8,
        return_tensors="pt",
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    return trainer

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(training_data)
cache_dir = "./"
trainer = trainer_fn(estimator_params={"train_dataset": train_dataset, "cache_dir": cache_dir})
trainer.train()

In [1]:
from typing import Dict, Any, List, Tuple
from sklearn.preprocessing import FunctionTransformer
from transformers import AutoTokenizer


def transformer_fn(transformer_params: Dict[str, Any]):
    """
    Returns an *unfitted* transformer that defines ``fit()`` and ``transform()`` methods.
    The transformer's input and output signatures should be compatible with scikit-learn
    transformers.
    """
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=transformer_params["cache_dir"],
        use_fast=True,
    )
    # Global parameters for tokenizer and model.
    max_seq_length = min(128, tokenizer.model_max_length)
    padding="max_length"
    question_column = "character"
    answer_column = "speech"
    
    def preprocess_squad_batch(
        examples,
        question_column: str,
        answer_column: str,
    ) -> Tuple[List[str], List[str]]:
        questions = examples[question_column]
        answers = examples[answer_column]
        pairs = [[q, a] for q, a in zip(questions, answers)]
        return questions, answers, pairs
    
    def preprocess_examples(examples):
        inputs, targets, pairs = preprocess_squad_batch(
            examples, question_column, answer_column
        )
        model_inputs = tokenizer(
            inputs,
            max_length=max_seq_length,
            padding=padding,
            truncation=True,
            return_tensors="pt",
        )
        labels = tokenizer(
            targets,
            max_length=max_seq_length,
            padding=padding,
            truncation=True,
            return_tensors="pt",
        )
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]
        model_inputs["labels"] = labels["input_ids"]
        model_inputs["decoder_input_ids"] = labels["input_ids"]
        return model_inputs

    return FunctionTransformer(preprocess_examples)

/Users/jin.zhang/Code/mlflow/.venvs/mlflow-dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trans = transformer_fn({"cache_dir": "./"})

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 8.36kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 483/483 [00:00<00:00, 122kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 226k/226k [00:00<00:00, 915kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 455k/455k [00:00<00:00, 980kB/s]


In [11]:
ingested_data = r.get_artifact("ingested_data")
ingested_data

,character,speech
0,First Citizen,"Before we proceed any further, hear me speak."
1,All,"Speak, speak."
2,First Citizen,You are all resolved rather to die than to fam...
3,All,Resolved. resolved.
4,First Citizen,"First, you know Caius Marcius is chief enemy t..."
...,...,...
7092,DUKE VINCENTIO,Nor I; my spirits are nimble.They fell togethe...
7093,Provost,"What, art thou waking?"
7094,ISABELLA,Do you not hear me speak?
7095,DUKE VINCENTIO,I do; and surelyIt is a sleepy language and th...


In [10]:
help(trans)

Help on FunctionTransformer in module sklearn.preprocessing._function_transformer object:

class FunctionTransformer(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  FunctionTransformer(func=None, inverse_func=None, *, validate=False, accept_sparse=False, check_inverse=True, feature_names_out=None, kw_args=None, inv_kw_args=None)
 |  
 |  Constructs a transformer from an arbitrary callable.
 |  
 |  A FunctionTransformer forwards its X (and optionally y) arguments to a
 |  user-defined function or function object and returns the result of this
 |  function. This is useful for stateless transformations such as taking the
 |  log of frequencies, doing custom scaling, etc.
 |  
 |  Note: If a lambda is used as the function, then the resulting
 |  transformer will not be pickleable.
 |  
 |  .. versionadded:: 0.17
 |  
 |  Read more in the :ref:`User Guide <function_transformer>`.
 |  
 |  Parameters
 |  ----------
 |  func : callable, default=None
 |      The callable to use

In [13]:
trans.transform(ingested_data["character"])

KeyError: 'character'